In [ ]:
from embedding_search.data_model import Author

author = Author.load("authors/0000-0002-4914-6671.json")
print(author)

In [ ]:
len(author.articles)

In [ ]:
len(author.articles_embeddings)

In [ ]:
from embedding_search.vector_store import MiniStore

s = MiniStore()

In [ ]:
s.add_author(author)

In [ ]:
s.metadata

In [ ]:
s.add_author

In [ ]:
import embedding_search

In [ ]:
embedding_search.data_model.Author

In [ ]:
embedding_search.data_model.Author

In [ ]:
from pathlib import Path
from embedding_search.orcid import Author
import numpy as np
import os

AUTHORS_DIR = Path("authors/")

metadatas = []
author_pickles = AUTHORS_DIR.glob("*.pickle")
for author_pickle in author_pickles:
    author = Author.load(author_pickle)
    embeddings = np.array(author.articles_embeddings, dtype=np.float32)

In [ ]:
author_pickles = AUTHORS_DIR.glob("*.pickle")
n = 0
for author_pickle in author_pickles:
    author = Author.load(author_pickle)
    n += len(author.articles_embeddings)
    print(
        f"Author: {author.first_name} {author.last_name} <{author.orcid}> ({len(author.articles_embeddings)} articles)"
    )

In [ ]:
doc_store.__dict__

In [ ]:
doc_store.add_texts()

In [ ]:
doc_store.index.get_item_vector(0)

In [ ]:
doc_store.index_to_docstore_id[0]

In [ ]:
doc_store.docstore._dict["710c0b72-b5b0-4bce-8f9c-0c879fc50c75"]

# Network graph

In [ ]:
!pip install matplotlib

In [ ]:
import altair as alt
import pandas as pd

# Create a DataFrame with the two points
data = pd.DataFrame({"x": [0, 1], "y": [0, 0.5]})

# Create the Altair chart with a line connecting the two points
chart = alt.Chart(data).mark_line().encode(x="x", y="y")

# Display the chart
chart

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create an empty graph
G = nx.Graph()

# Add author nodes
G.add_nodes_from([1, 2, 3, 4, 5])
node_colors = ["lightblue" for _ in range(5)]

# Add article nodes
G.add_nodes_from([6, 7, 8])
node_colors.extend(["red" for _ in range(3)])

# Add edges
G.add_edges_from([(1, 6), (1, 7), (2, 8), (3, 6)])

# Plot the graph


nx.draw(G, with_labels=True, node_color=node_colors, edge_color="gray")

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import altair as alt

nodes = ['a', 'b', 'c']
edges = 

df_nodes = pd.DataFrame(nodes, columns=["id"])
df_edges = pd.DataFrame(edges, columns=["source", "target"])
    
# Create an Altair chart
chart = alt.Chart(df_edges).mark_circle(size=200).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None),
    color=alt.Color('source:N', legend=None)
)

# Add tooltips to the chart
chart = chart.encode(
    tooltip=['source', 'target']
)

# Add force layout transform to the chart
chart = chart.transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=df_nodes, key='id', fields=['x', 'y'])
).transform_force(
    force='center',
    x='x',
    y='y'
)

# Set the chart size and remove axis
chart = chart.properties(width=600, height=400).configure_axis(
    grid=False,
    ticks=False,
    labels=False
)

chart